In [ ]:
import requests
import json
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime
from dotenv import load_dotenv

# Carregar ambiente
load_dotenv('../.env.local')

# Configuração
BACKEND_URL = "http://localhost:3001"
API_URL = f"{BACKEND_URL}/api"
DATABASE_URL = os.getenv("DATABASE_URL")

def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print(f"✅ Configurado para testar: {BACKEND_URL}")
print(f"🗄️  PostgreSQL: {'Conectado' if DATABASE_URL else 'Não configurado'}")

## 1. Health Check
Verifica se o backend está online e respondendo.

In [ ]:
try:
    response = requests.get(f"{BACKEND_URL}/health", timeout=5)
    print(f"Status Code: {response.status_code}")
    print_json(response.json())

    # Assertions
    assert response.status_code == 200, f"Expected 200, got {response.status_code}"
    data = response.json()
    assert data.get('status') == 'ok' or data.get('status') == 'online', "Status should be ok/online"
    print("✅ Health Check Passed")
except AssertionError as ae:
    print(f"❌ Assertion Failed: {ae}")
except Exception as e:
    print(f"❌ Erro ao conectar: {e}")

## 2. Teste de IA (Gemini)
Envia um prompt para o endpoint de streaming da IA. Nota: Este teste usa o endpoint REST padrão, não o streaming SSE, para simplificar a visualização.

In [ ]:
payload = {
    "messages": [
        {"role": "system", "content": "Você é um assistente jurídico útil."},
        {"role": "user", "content": "Resuma o que é um Habeas Corpus em uma frase."}
    ],
    "temperature": 0.7
}

print("🤖 Enviando requisição para IA...")
try:
    # Usando stream=True para SSE
    with requests.post(f"{API_URL}/llm-stream", json=payload, stream=True, timeout=30) as r:
        assert r.status_code == 200, f"Expected 200, got {r.status_code}"
        print(f"Status: {r.status_code}")

        full_response = ""
        content_received = False

        for line in r.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith('data: '):
                    data_str = decoded_line[6:]
                    try:
                        data_json = json.loads(data_str)
                        if data_json.get('type') == 'content':
                            content = data_json.get('content', '')
                            print(content, end='', flush=True)
                            full_response += content
                            content_received = True
                        elif data_json.get('type') == 'error':
                            print(f"\n❌ Erro da IA: {data_json.get('message')}")
                    except:
                        pass

        print("\n")
        assert content_received, "No content received from AI stream"
        assert len(full_response) > 10, "Response too short"
        print("✅ IA Test Passed")

except AssertionError as ae:
    print(f"❌ Assertion Failed: {ae}")
except Exception as e:
    print(f"❌ Erro: {e}")

## 3. Teste KV Store (Redis/Memória)
Testa a persistência de dados.

In [ ]:
# 1. Salvar um valor
test_key = "teste-notebook-integration"
test_value = {"data": str(datetime.now()), "autor": "Notebook Test", "check": "valid"}

print(f"💾 Salvando chave '{test_key}'...")
try:
    res_post = requests.post(f"{API_URL}/kv/{test_key}", json={"value": test_value}, timeout=5)
    assert res_post.status_code in [200, 201], f"Post failed: {res_post.status_code}"
    print_json(res_post.json())

    # 2. Recuperar o valor
    print(f"\n📂 Recuperando chave '{test_key}'...")
    res_get = requests.get(f"{API_URL}/kv/{test_key}", timeout=5)
    assert res_get.status_code == 200, f"Get failed: {res_get.status_code}"
    data = res_get.json()
    print_json(data)

    # Assertions
    # A estrutura de retorno depende da implementação do backend (pode ser data['value'] ou direto)
    # Ajuste conforme necessário. Assumindo que retorna o objeto salvo.
    val_retrieved = data.get('value') if 'value' in data else data
    # Se o backend encapsula em 'result' ou similar, ajuste aqui.

    # Cleanup (se houver endpoint DELETE, senão apenas log)
    # requests.delete(f"{API_URL}/kv/{test_key}")

    print("✅ KV Store Test Passed")

except AssertionError as ae:
    print(f"❌ Assertion Failed: {ae}")
except Exception as e:
    print(f"❌ Erro: {e}")

## 4. Listar Agentes
Verifica quais agentes estão disponíveis no sistema.

In [ ]:
try:
    response = requests.get(f"{API_URL}/agents/list", timeout=5)
    if response.status_code == 200:
        agents = response.json()
        print(f"✅ Encontrados {len(agents)} agentes:")
        for agent in agents:
            print(f" - {agent.get('name')} ({agent.get('id')})")

        assert len(agents) > 0, "No agents found"
        assert 'id' in agents[0], "Agent missing ID"
        assert 'name' in agents[0], "Agent missing Name"
        print("✅ Agents List Test Passed")
    else:
        print(f"⚠️ Status inesperado: {response.status_code}")
        print(response.text)
        assert False, f"API returned {response.status_code}"
except AssertionError as ae:
    print(f"❌ Assertion Failed: {ae}")
except Exception as e:
    print(f"❌ Erro: {e}")

## 5. Teste de Conexão PostgreSQL
Valida a conexão direta com o banco de dados.

In [ ]:
if DATABASE_URL:
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor(cursor_factory=RealDictCursor)

        # Test query
        cursor.execute("SELECT version()")
        version = cursor.fetchone()['version']
        print(f"✅ PostgreSQL conectado: {version[:50]}...")
        assert "PostgreSQL" in version, "Not a PostgreSQL database"

        # Verificar minutas
        cursor.execute("SELECT COUNT(*) as total FROM minutas")
        total = cursor.fetchone()['total']
        print(f"📄 Total de minutas: {total}")
        assert total >= 0, "Total count invalid"

        cursor.close()
        conn.close()
        print("✅ Database Test Passed")
    except AssertionError as ae:
        print(f"❌ Assertion Failed: {ae}")
    except Exception as e:
        print(f"❌ Erro: {e}")
else:
    print("⚠️ DATABASE_URL não configurada")
    # assert False, "DATABASE_URL missing" # Opcional: falhar se não houver DB

: 